In [1]:
# Importing libraries neeeded for the project
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
import numpy

import warnings
from math import sqrt


In [2]:
#!/usr/bin/env /Applications/MAMP/Library/bin/python

import mysql.connector

config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'ecotourisme',
  'raise_on_warnings': True
}

cnx = mysql.connector.connect(**config)

cursor = cnx.cursor(dictionary=True)

cursor.execute('SELECT * FROM `ecologie` where id_pays = "US"')
results = cursor.fetchall()
cnx.close()

data1 = pd.DataFrame(results)

data = data1
# afficher  par ordre decroissant annee
data = data.sort_values(by='annee', ascending=True) 
data.head()


,id,id_pays,annee,co2,ges,elecRenew,GES_hab
0,198,US,1990,4844517.4,5855541.470,10.917163,23.7677
1,409,US,1991,4807496.5,5810376.768,10.741865,23.5844
2,620,US,1992,4879626.1,5894661.485,9.729862,23.9265
3,831,US,1993,4995209.5,6006008.717,10.270782,24.3784
4,1042,US,1994,5066803.0,6100512.901,9.532383,24.7620


In [3]:
data = data.set_index('annee')
series = data[['co2']]

Recherche des bons parametre ARIMA(p,d,q)

In [4]:
X = series.values

In [5]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.8)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	rmse = sqrt(mean_squared_error(test, predictions))
	return rmse

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))


p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(series.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) RMSE=520546.965
ARIMA(0, 0, 1) RMSE=336693.326
ARIMA(0, 0, 2) RMSE=292459.776
ARIMA(0, 1, 0) RMSE=215485.713
ARIMA(0, 1, 1) RMSE=215823.408
ARIMA(0, 1, 2) RMSE=216634.305
ARIMA(0, 2, 0) RMSE=206955.831
ARIMA(0, 2, 1) RMSE=207617.105
ARIMA(0, 2, 2) RMSE=220253.768
ARIMA(1, 0, 0) RMSE=233870.015
ARIMA(1, 0, 1) RMSE=236650.064
ARIMA(1, 0, 2) RMSE=227729.854
ARIMA(1, 1, 0) RMSE=215835.566
ARIMA(1, 1, 1) RMSE=215950.706
ARIMA(1, 1, 2) RMSE=214091.613
ARIMA(1, 2, 0) RMSE=207659.251
ARIMA(1, 2, 1) RMSE=202525.543
ARIMA(1, 2, 2) RMSE=201620.751
ARIMA(2, 0, 0) RMSE=235326.295
ARIMA(2, 0, 1) RMSE=231663.035
ARIMA(2, 0, 2) RMSE=237979.184
ARIMA(2, 1, 0) RMSE=216580.051
ARIMA(2, 1, 1) RMSE=214225.884
ARIMA(2, 1, 2) RMSE=201088.750
ARIMA(2, 2, 0) RMSE=204268.400
ARIMA(2, 2, 1) RMSE=210730.299
ARIMA(2, 2, 2) RMSE=246840.922
ARIMA(4, 0, 0) RMSE=236108.251
ARIMA(4, 0, 1) RMSE=223384.504
ARIMA(4, 0, 2) RMSE=234728.779
ARIMA(4, 1, 0) RMSE=214525.496
ARIMA(4, 1, 1) RMSE=211673.187
ARIMA(4,